In [2]:
# Built-in packages-----------------
import os
import sys
import shutil
from typing import Optional

# Filesystem management-----------------
import earthaccess

import fsspec

from obstore.auth.earthdata import NasaEarthdataCredentialProvider
from obstore.store import S3Store
from obspec_utils.registry import ObjectStoreRegistry

# Data handling-----------------
import numpy as np
import xarray as xr
from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
import pandas as pd

## 1. Get Data File S3 endpoints in Earthdata Cloud

In [3]:
# Get Earthdata creds
auth = earthaccess.login()

In [4]:
# Locate file information / metadata:
shortname = "OSTIA-UKMO-L4-GLOB-REP-v2.0"
granule_info = earthaccess.search_data(
    short_name=shortname,
    )

In [6]:
# Get S3 endpoints for all files:
#data_s3links = [g.data_links(access="direct")[0] for g in granule_info]
data_links = [g.data_links()[0] for g in granule_info]
print("Number of granules found =", len(data_links))
print("First few granules:")
data_links[0:5]

Number of granules found = 15340
First few granules:


['https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/001/19820101120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/002/19820102120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/003/19820103120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/004/19820104120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc',
 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/005/19820105120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc']

## 2. Generate a single-day reference file

In [7]:
credentials_url = "https://archive.podaac.earthdata.nasa.gov/s3credentials"
bucket = "s3://podaac-ops-cumulus-protected"

In [13]:
credential_provider = NasaEarthdataCredentialProvider(credentials_url, auth=(auth.username, auth.password))
store = S3Store.from_url(bucket, credential_provider=credential_provider)
registry = ObjectStoreRegistry({bucket: store})

In [14]:
%%time
vds_ref_single = open_virtual_dataset(
    data_links[0],
    parser=HDFParser(),
    registry=registry,
    loadable_variables=['lat', 'lon', 'time'],
    decode_times=False
)

CPU times: user 789 μs, sys: 1.05 ms, total: 1.84 ms
Wall time: 1.06 ms


ValueError: Could not find an ObjectStore matching the url `https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OSTIA-UKMO-L4-GLOB-REP-v2.0/1982/001/19820101120000-UKMO-L4_GHRSST-SSTfnd-OSTIA-GLOB_REP-v02.0-fv02.0.nc`